In [16]:
import numpy as np
import pandas as pd
import datetime
import time
import os
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
from sklearn.ensemble import RandomForestRegressor


print(os.listdir("../machine_learning"))

['.git', '.gitignore', '.ipynb_checkpoints', 'data_base', 'Evaluation Truncated Model Neural Network.ipynb', 'get-pip.py', 'Include', 'Lib', 'ML_Earthquake_Neuron-FullDataSet.ipynb', 'ML_Earthquake_Neuron_Trained with 20k.ipynb', 'ML_Earthquake_Neuron_Trained with Truncated Data with Months.ipynb', 'ML_Earthquake_Neuron_Trained with Truncated Data.ipynb', 'ML_Evaluating_Model.ipynb', 'ml_kellog_db.ipynb', 'models', 'pyvenv.cfg', 'README.md', 'Scripts']


In [17]:
# Loading the Data

data = pd.read_csv("data_base/USGS_earthquakes/earthquakes_truncated_month_testing_10k.csv")
data.head()

,month,latitude,longitude,depth,mag,id,place
0,1,-60,0,15.0,6.3,iscgem877909,Bouvet Island region
1,1,40,90,15.0,5.7,iscgem877920,"northern Xinjiang, China"
2,1,10,40,15.0,6.1,iscgemsup877930,Djibouti
3,1,50,30,40.0,5.4,iscgemsup877933,Romania
4,1,10,100,15.0,5.6,iscgem877954,"Nicobar Islands, India region"


In [18]:
data = data[['month','latitude', 'longitude', 'depth', 'mag']]
data.head()

,month,latitude,longitude,depth,mag
0,1,-60,0,15.0,6.3
1,1,40,90,15.0,5.7
2,1,10,40,15.0,6.1
3,1,50,30,40.0,5.4
4,1,10,100,15.0,5.6


In [19]:
#Displaying the Columns


#After you've removed day and year

mo_cat = CategoricalDtype(categories=[str(i) for i in range(1,13)], ordered=True)
data.month.astype(mo_cat)

data.columns


Index(['month', 'latitude', 'longitude', 'depth', 'mag'], dtype='object')

In [20]:
data.month.dtype

dtype('int64')

In [21]:
print(mo_cat)

category


Model Splitting 

In [22]:
#Splitting the data

X = data [['month','latitude', 'longitude']]
y = data [['depth', 'mag']]

In [23]:
#Splitting Xs and ys into traing and test datasets 

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

(8117, 3) (2030, 3) (8117, 2) (2030, 3)


In [24]:
#from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train)
reg.predict(X_test)



array([[158.92082351,   5.79144977],
       [ 13.4675    ,   5.63996429],
       [ 25.364     ,   5.69995   ],
       ...,
       [ 86.23392134,   5.54198175],
       [ 54.26361253,   5.56851059],
       [111.27799968,   5.67186367]])

In [25]:
# Print the Features that are going to be tested into the model

print(X_test)

      month  latitude  longitude
9720      9       -10        130
3110     10        80         10
5992      6       -60        150
5894      5       -10        110
2615      4       -30        -70
...     ...       ...        ...
6060      6       -10        130
7986     10       -10        120
3194      1        50        150
4843      8       -30        -70
1739      2        10        120

[2030 rows x 3 columns]


In [26]:
# Calculating R Score

reg.score(X_test, y_test)


0.18751478439799596

In [27]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':[10, 20, 50, 100, 200, 500]}

grid_obj = GridSearchCV(reg, parameters)
grid_fit = grid_obj.fit(X_train, y_train)
best_fit = grid_fit.best_estimator_
best_fit.predict(X_test)

array([[155.36868681,   5.80838752],
       [ 13.624     ,   5.62098286],
       [ 24.86985714,   5.72313381],
       ...,
       [ 83.07166934,   5.52086845],
       [ 53.88970076,   5.56657143],
       [107.61415022,   5.69655147]])

In [28]:
#Creating the Best Fit 

best_fit.score(X_test, y_test)


0.1878185758101672

##Neural Network Model

In [29]:
#Creating the Neural Model

from keras.models import Sequential
from keras.layers import Dense

def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [30]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, verbose=0)

# neurons = [16, 64, 128, 256]
neurons = [16]
# batch_size = [10, 20, 50, 100]
batch_size = [10]
epochs = [10]
# activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'exponential']
activation = ['sigmoid', 'relu']
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizer = ['SGD', 'Adadelta']
loss = ['squared_hinge']

param_grid = dict(neurons=neurons, batch_size=batch_size, epochs=epochs, activation=activation, optimizer=optimizer, loss=loss)

In [31]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.717129 using {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.394906 (0.378436) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.533252 (0.445062) with: {'activation': 'sigmoid', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}
0.717129 (0.080543) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'SGD'}
0.187278 (0.217716) with: {'activation': 'relu', 'batch_size': 10, 'epochs': 10, 'loss': 'squared_hinge', 'neurons': 16, 'optimizer': 'Adadelta'}


In [32]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(3,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, batch_size=125, epochs=900, verbose=1, validation_data=(X_test, y_test))

Epoch 1/900
65/65 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.6724 - val_loss: 0.0036 - val_accuracy: 0.6704
Epoch 2/900
65/65 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.6746 - val_loss: 0.0036 - val_accuracy: 0.7089
Epoch 3/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6757 - val_loss: 0.0036 - val_accuracy: 0.7054
Epoch 4/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6952 - val_loss: 0.0036 - val_accuracy: 0.6980
Epoch 5/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6788 - val_loss: 0.0036 - val_accuracy: 0.6956
Epoch 6/900
65/65 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.6750 - val_loss: 0.0036 - val_accuracy: 0.6685
Epoch 7/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6818 - val_loss: 0.0036 - val_accuracy: 0.6621
Epoch 8/900
6

Epoch 59/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6823 - val_loss: 0.0036 - val_accuracy: 0.6734
Epoch 60/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6952 - val_loss: 0.0036 - val_accuracy: 0.6768
Epoch 61/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6894 - val_loss: 0.0036 - val_accuracy: 0.6980
Epoch 62/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6984 - val_loss: 0.0036 - val_accuracy: 0.7049
Epoch 63/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6999 - val_loss: 0.0036 - val_accuracy: 0.6970
Epoch 64/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6883 - val_loss: 0.0036 - val_accuracy: 0.7084
Epoch 65/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7030 - val_loss: 0.0036 - val_accuracy: 0.7010
Epoch 

65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6948 - val_loss: 0.0035 - val_accuracy: 0.7305
Epoch 117/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6708 - val_loss: 0.0036 - val_accuracy: 0.6443
Epoch 118/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6884 - val_loss: 0.0036 - val_accuracy: 0.7099
Epoch 119/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6951 - val_loss: 0.0035 - val_accuracy: 0.6828
Epoch 120/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7044 - val_loss: 0.0036 - val_accuracy: 0.6729
Epoch 121/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6775 - val_loss: 0.0036 - val_accuracy: 0.6773
Epoch 122/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6941 - val_loss: 0.0035 - val_accuracy: 0.7222
Epoch 123/900

65/65 [==============================] - 0s 5ms/step - loss: 0.0032 - accuracy: 0.6866 - val_loss: 0.0035 - val_accuracy: 0.7374
Epoch 174/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6924 - val_loss: 0.0036 - val_accuracy: 0.6833
Epoch 175/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6995 - val_loss: 0.0035 - val_accuracy: 0.6828
Epoch 176/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6946 - val_loss: 0.0035 - val_accuracy: 0.6970
Epoch 177/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6773 - val_loss: 0.0036 - val_accuracy: 0.6685
Epoch 178/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.6850 - val_loss: 0.0036 - val_accuracy: 0.7384
Epoch 179/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7085 - val_loss: 0.0035 - val_accuracy: 0.7232
Epoch 180/900

65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7116 - val_loss: 0.0036 - val_accuracy: 0.6892
Epoch 231/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7106 - val_loss: 0.0036 - val_accuracy: 0.6906
Epoch 232/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7179 - val_loss: 0.0036 - val_accuracy: 0.7300
Epoch 233/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7162 - val_loss: 0.0035 - val_accuracy: 0.7399
Epoch 234/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7072 - val_loss: 0.0035 - val_accuracy: 0.7315
Epoch 235/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7216 - val_loss: 0.0036 - val_accuracy: 0.6966
Epoch 236/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7116 - val_loss: 0.0035 - val_accuracy: 0.7010
Epoch 237/900

...

65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7248 - val_loss: 0.0035 - val_accuracy: 0.7305
Epoch 630/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7329 - val_loss: 0.0035 - val_accuracy: 0.7315
Epoch 631/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7375 - val_loss: 0.0036 - val_accuracy: 0.7330
Epoch 632/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7290 - val_loss: 0.0036 - val_accuracy: 0.6995
Epoch 633/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7259 - val_loss: 0.0035 - val_accuracy: 0.7286
Epoch 634/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7338 - val_loss: 0.0035 - val_accuracy: 0.7256
Epoch 635/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7235 - val_loss: 0.0035 - val_accuracy: 0.7675
Epoch 636/900

65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7351 - val_loss: 0.0035 - val_accuracy: 0.7330
Epoch 687/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7339 - val_loss: 0.0035 - val_accuracy: 0.7217
Epoch 688/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7271 - val_loss: 0.0035 - val_accuracy: 0.7394
Epoch 689/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7490 - val_loss: 0.0035 - val_accuracy: 0.7350
Epoch 690/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7346 - val_loss: 0.0035 - val_accuracy: 0.7315
Epoch 691/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7468 - val_loss: 0.0036 - val_accuracy: 0.7236
Epoch 692/900
65/65 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.7392 - val_loss: 0.0036 - val_accuracy: 0.7360
Epoch 693/900

65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7419 - val_loss: 0.0035 - val_accuracy: 0.7384
Epoch 744/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7540 - val_loss: 0.0035 - val_accuracy: 0.7493
Epoch 745/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7509 - val_loss: 0.0035 - val_accuracy: 0.7232
Epoch 746/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7415 - val_loss: 0.0036 - val_accuracy: 0.7355
Epoch 747/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7433 - val_loss: 0.0035 - val_accuracy: 0.7379
Epoch 748/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7471 - val_loss: 0.0035 - val_accuracy: 0.7369
Epoch 749/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7442 - val_loss: 0.0035 - val_accuracy: 0.7768
Epoch 750/900

65/65 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.7376 - val_loss: 0.0035 - val_accuracy: 0.7379
Epoch 801/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7425 - val_loss: 0.0035 - val_accuracy: 0.7463
Epoch 802/900
65/65 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.7452 - val_loss: 0.0035 - val_accuracy: 0.7567
Epoch 803/900
65/65 [==============================] - 0s 3ms/step - loss: 0.0032 - accuracy: 0.7458 - val_loss: 0.0035 - val_accuracy: 0.7414
Epoch 804/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7440 - val_loss: 0.0035 - val_accuracy: 0.7414
Epoch 805/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7543 - val_loss: 0.0035 - val_accuracy: 0.7414
Epoch 806/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7482 - val_loss: 0.0035 - val_accuracy: 0.7808
Epoch 807/900

Epoch 857/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7582 - val_loss: 0.0035 - val_accuracy: 0.7857
Epoch 858/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7521 - val_loss: 0.0035 - val_accuracy: 0.7788
Epoch 859/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7476 - val_loss: 0.0035 - val_accuracy: 0.7463
Epoch 860/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7587 - val_loss: 0.0035 - val_accuracy: 0.7828
Epoch 861/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7606 - val_loss: 0.0035 - val_accuracy: 0.7842
Epoch 862/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7596 - val_loss: 0.0035 - val_accuracy: 0.7557
Epoch 863/900
65/65 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 0.7558 - val_loss: 0.0035 - val_accuracy: 0.7562

In [34]:
[test_loss, test_acc] = model.evaluate(X_test, y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

64/64 [==============================] - 0s 2ms/step - loss: 0.0036 - accuracy: 0.5734
Evaluation result on Test Data : Loss = 0.0036166238132864237, accuracy = 0.573399007320404


In [ ]:
model.save('models/ml_model_truncated_month_data.h5')

# Loading a Model

In [ ]:
# Load the model
# from tensorflow.keras.models import load_model
# eq_model = load_model("models/ml_model_fulldata.h5")
import matplotlib.pyplot as plt



plt.plot(model.predict([[1,-x,-70] for x in range(10,100, 10)]))